In [1]:
! pip install mlxtend

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
from collections import defaultdict
import scipy.stats as stat
import importlib


import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler


from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [73]:
df_train=pd.read_csv("house_price_train.csv")

In [74]:
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train.drop('SalePrice',axis=1,inplace=True)


In [75]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [76]:
df_train.describe()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [77]:
df_train.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 79, dtype: int64

In [78]:
# Columns which have zero values in more than 50% rows are removed
def column_with_most_reduntant_values(reduntant_value,df_train:pd.DataFrame,threshold:float):
    reduntant_counts={}
    columns_with_most_reduntant_counts=[]
    for col in df_train.columns:
        if df_train[col].dtypes=='int' or df_train[col].dtypes=='float':
            for values in df_train[col]:
                if values==reduntant_value:
                    reduntant_counts[col]=1+reduntant_counts.get(col,0)
    for col in reduntant_counts:
        if reduntant_counts[col]>=(threshold*len(df_train)):
            columns_with_most_reduntant_counts.append(col)
    df_train.drop(columns_with_most_reduntant_counts,axis=1,inplace=True)
        
        
            

In [79]:
column_with_most_reduntant_values(reduntant_value=0,df_train=df_train,threshold=0.5)

In [80]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,GarageCond,PavedDrive,OpenPorchSF,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,61,NaN,NaN,NaN,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,TA,Y,0,NaN,NaN,NaN,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,TA,Y,42,NaN,NaN,NaN,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,TA,Y,35,NaN,NaN,NaN,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,TA,Y,84,NaN,NaN,NaN,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,40,NaN,NaN,NaN,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,0,NaN,MnPrv,NaN,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,60,NaN,GdPrv,Shed,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,0,NaN,NaN,NaN,4,2010,WD,Normal


In [81]:
# Deleting columns which have more than 50% NULL values

def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)
   


In [82]:
# print(column_with_most_reduntant_null_values(threshold=0.5,X=df_train))

In [83]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [84]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [85]:
i=0
for cols in df_train.columns:
    if df_train[cols].dtype=="int64":
        # print(cols,i)
        i+=1
    elif df_train[cols].dtype=="object":
#         print(cols,i)
        i+=1
#     elif df_train[cols].dtype=="float64":
#         print(cols,i)
# #         i+=1
print(i)

60


In [86]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [87]:
# Filling Null Values

def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)


In [88]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [89]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [90]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [91]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [92]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [93]:
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [94]:
df_train["MSZoning"].dtypes

dtype('O')

In [95]:
# fig, axes = plt.subplots(nrows = 6, ncols = 4)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(40, 40)

# for ax, col in zip(axes, continuous_cols):
#     print()
#     sns.distplot(df_train[col], ax = ax,kde=True)
#     ax.set_title(col)

In [96]:
# fig, axes = plt.subplots(nrows = 6, ncols = 4)#   skew axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(40, 40)

# for ax, col in zip(axes, continuous_cols):
#     # print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [97]:
# plt.figure(figsize=(20,15))
# sns.heatmap(df_train.loc[:,continuous_cols].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [98]:
# sns.set(rc={'figure.figsize':(40,40)})
# cData_attr = df_train.loc[:,continuous_cols]
# sns.pairplot(cData_attr, diag_kind='kde')  

In [99]:
df_train.loc[:,continuous_cols]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,706,150,856,...,3,1,8,0,2003.0,2,548,61,2,2008
1,20,80.0,9600,6,8,1976,1976,978,284,1262,...,3,1,6,1,1976.0,2,460,0,5,2007
2,60,68.0,11250,7,5,2001,2002,486,434,920,...,3,1,6,1,2001.0,2,608,42,9,2008
3,70,60.0,9550,7,5,1915,1970,216,540,756,...,3,1,7,1,1998.0,3,642,35,2,2006
4,60,84.0,14260,8,5,2000,2000,655,490,1145,...,4,1,9,1,2000.0,3,836,84,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0,953,953,...,3,1,7,1,1999.0,2,460,40,8,2007
1456,20,85.0,13175,6,6,1978,1988,790,589,1542,...,3,1,7,2,1978.0,2,500,0,2,2010
1457,70,66.0,9042,7,9,1941,2006,275,877,1152,...,4,1,9,2,1941.0,1,252,60,5,2010
1458,20,68.0,9717,5,6,1950,1996,49,0,1078,...,2,1,5,0,1950.0,1,240,0,4,2010


In [100]:
continuous_cols

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'FullBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'OpenPorchSF',
 'MoSold',
 'YrSold']

In [101]:
df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here


In [102]:

for col in continuous_cols:
    q1=df_train[col].quantile(0.25)
    q3=df_train[col].quantile(0.75)
    iqr=q3-q1
    l=q1-1.5*iqr
    h=q3+1.5*iqr
    print("[",l,",",h,"]")
    df_train = df_train[(df_train[col] <= h)]# & (df_train[col] >=l)] 
    df_train =df_train[(df_train[col] >=l)] 
   




[ -55.0 , 145.0 ]
[ 30.0 , 110.0 ]
[ 2440.0 , 16936.0 ]
[ 2.0 , 10.0 ]
[ 3.5 , 7.5 ]
[ 1887.0 , 2071.0 ]
[ 1909.0 , 2061.0 ]
[ -1099.125 , 1831.875 ]
[ -636.375 , 1718.625 ]
[ 141.25 , 1983.25 ]
[ 207.25 , 2053.25 ]
[ 236.25 , 2622.25 ]
[ -0.5 , 3.5 ]
[ 0.5 , 4.5 ]
[ 1.0 , 1.0 ]
[ 2.0 , 10.0 ]
[ -1.5 , 2.5 ]
[ 1904.5 , 2060.5 ]
[ -0.5 , 3.5 ]
[ -11.5 , 928.5 ]
[ -102.0 , 170.0 ]
[ -2.0 , 14.0 ]
[ 2004.0 , 2012.0 ]


In [103]:
output_col=df_train['SalePrice']
df_train.drop("SalePrice",axis=1,inplace=True)

In [104]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
5,50,RL,85.0,14115,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,480,TA,TA,Y,30,10,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,20,RL,78.0,9262,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,3,840,TA,TA,Y,36,5,2009,New,Partial
1454,20,FV,62.0,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,400,TA,TA,Y,113,10,2009,WD,Normal
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [105]:
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [106]:


def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train
        

    
    


In [107]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="labelencoder")

In [108]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,60,65.0,8450,7,5,2003,2003,706,150,856,...,1,0,0,0,1,0,0,0,0,0
2,60,68.0,11250,7,5,2001,2002,486,434,920,...,1,0,1,0,1,0,0,0,0,0
3,70,60.0,9550,7,5,1915,1970,216,540,756,...,1,0,0,1,0,0,0,0,0,2
4,60,84.0,14260,8,5,2000,2000,655,490,1145,...,1,0,1,0,1,0,0,0,0,0
5,50,85.0,14115,5,5,1993,1995,732,64,796,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,20,78.0,9262,8,5,2008,2009,0,1573,1573,...,2,0,0,0,2,0,0,0,1,1
1454,20,62.0,7500,7,5,2004,2005,410,811,1221,...,1,0,0,0,1,0,0,0,0,0
1455,60,62.0,7917,6,5,1999,2000,0,953,953,...,0,0,1,0,1,0,0,0,0,0
1458,20,68.0,9717,5,6,1950,1996,49,0,1078,...,1,0,0,0,0,0,0,0,0,0


In [109]:
for col in categorical_cols:
    if col in set(df_train.columns):
        print(col)

MSZoning
Street
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
SaleType
SaleCondition


In [110]:
df_train.loc[:,continuous_cols].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,MoSold,YrSold
count,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,...,953.000000,953.0,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000
mean,47.675761,68.954879,9337.533054,6.111228,5.498426,1974.242392,1984.696747,445.371459,567.120672,1056.137461,...,2.790136,1.0,6.253935,0.581322,1979.776495,1.754460,467.279119,36.126967,6.265477,2007.809024
std,30.136712,14.082256,2665.346804,1.249458,0.790069,28.708602,20.807273,405.964996,404.340239,313.161547,...,0.661062,0.0,1.291759,0.592271,24.089979,0.678716,186.486093,44.027298,2.722424,1.332860
min,20.000000,30.000000,2887.000000,2.000000,4.000000,1890.000000,1950.000000,0.000000,0.000000,264.000000,...,1.000000,1.0,3.000000,0.000000,1906.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,60.000000,7758.000000,5.000000,5.000000,1956.000000,1965.000000,0.000000,255.000000,836.000000,...,2.000000,1.0,5.000000,0.000000,1963.000000,1.000000,336.000000,0.000000,4.000000,2007.000000
50%,50.000000,69.000000,9240.000000,6.000000,5.000000,1977.000000,1994.000000,410.000000,486.000000,1004.000000,...,3.000000,1.0,6.000000,1.000000,1981.000000,2.000000,476.000000,20.000000,6.000000,2008.000000
75%,60.000000,78.000000,10998.000000,7.000000,6.000000,2002.000000,2004.000000,713.000000,801.000000,1258.000000,...,3.000000,1.0,7.000000,1.000000,2002.000000,2.000000,576.000000,62.000000,8.000000,2009.000000
max,120.000000,110.000000,16770.000000,10.000000,7.000000,2009.000000,2010.000000,1646.000000,1710.000000,1976.000000,...,4.000000,1.0,10.000000,2.000000,2010.000000,3.000000,928.000000,170.000000,12.000000,2010.000000


In [111]:
### SCALING

In [112]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns,)
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,0.4,0.4375,0.400706,0.625,0.333333,0.949580,0.883333,0.428919,0.087719,0.345794,...,0.333333,0.0,0.00,0.0,0.5,0.0,0.0,0.0,0.000,0.0
1,0.4,0.4750,0.602391,0.625,0.333333,0.932773,0.866667,0.295261,0.253801,0.383178,...,0.333333,0.0,0.25,0.0,0.5,0.0,0.0,0.0,0.000,0.0
2,0.5,0.3750,0.479939,0.625,0.333333,0.210084,0.333333,0.131227,0.315789,0.287383,...,0.333333,0.0,0.00,0.2,0.0,0.0,0.0,0.0,0.000,0.4
3,0.4,0.6750,0.819203,0.750,0.333333,0.924370,0.833333,0.397934,0.286550,0.514603,...,0.333333,0.0,0.25,0.0,0.5,0.0,0.0,0.0,0.000,0.0
4,0.3,0.6875,0.808759,0.375,0.333333,0.865546,0.750000,0.444714,0.037427,0.310748,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,0.0,0.6000,0.459195,0.750,0.333333,0.991597,0.983333,0.000000,0.919883,0.764603,...,0.666667,0.0,0.00,0.0,1.0,0.0,0.0,0.0,0.125,0.2
949,0.0,0.4000,0.332277,0.625,0.333333,0.957983,0.916667,0.249089,0.474269,0.558995,...,0.333333,0.0,0.00,0.0,0.5,0.0,0.0,0.0,0.000,0.0
950,0.4,0.4000,0.362314,0.500,0.333333,0.915966,0.833333,0.000000,0.557310,0.402453,...,0.000000,0.0,0.25,0.0,0.5,0.0,0.0,0.0,0.000,0.0
951,0.0,0.4750,0.491969,0.375,0.666667,0.504202,0.766667,0.029769,0.000000,0.475467,...,0.333333,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0


In [113]:
from sklearn.metrics import r2_score

In [124]:
X=df_train
Y=output_col

In [125]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=None,shuffle=True)


In [126]:
accuracies_with_sfs={}
accuracies_with_hyperparameter_tuning={}

In [127]:
def hyperparameter_tuning(algo:str,param_grid:dict,n_iter:int,cv:int,verbose:int,random_state:int,X_train,Y_train,X_test,Y_test,evaluation_metric:str):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    
    model_random_cv=RandomizedSearchCV(estimator=model,param_distributions=param_grid,n_iter=n_iter,cv=cv,verbose=verbose,random_state=random_state,n_jobs=-1)
    model_random_cv.fit(X_train,Y_train)
    best_params_=model_random_cv.best_params_
    print(f"{model} RandomCV Best Params :",best_params_)
    best_random_grid=model_random_cv.best_estimator_
    y_pred=best_random_grid.predict(X_test)
    
    score=evaluation_metric(Y_test,y_pred)
    print(f"{model} RandomCV Score:",score)
    
    
    
    param_grid=defaultdict(list)
    for val in best_params_:
        param_grid[val].append(best_params_[val])
    
    
    model_for_gcv=eval(algo)
    
    grid_search=GridSearchCV(estimator=model_for_gcv,param_grid=param_grid,cv=cv,n_jobs=-1,verbose=verbose)
    grid_search.fit(X_train,Y_train)
    best_grid2=grid_search.best_estimator_
    y_pred2=best_grid2.predict(X_test)
    score_1=evaluation_metric(Y_test,y_pred2)
    print("Final score is ",score_1)
    accuracies_with_hyperparameter_tuning[algo]=score_1

In [128]:
def sf_selector(algo:str,forward:bool,floating:bool,scoring:int,X_train,Y_train,X_test,Y_test,min_featues:int,cv:int,verbose:int):
    model=eval(algo)
    sfs=SFS(model, 
           k_features=(min_featues,len(X_train.columns)), 
           forward=forward, 
           floating=floating, 
           verbose=verbose,
           scoring=scoring,
           cv=cv,
           n_jobs=-1)
    sfs=sfs.fit(X_train, Y_train)
    print("Selected Features",sfs.k_feature_names_)
    model_=eval(algo)
    model_.fit(X_train.loc[:,list(sfs.k_feature_names_)],Y_train)
    score=model_.score(X_test.loc[:,list(sfs.k_feature_names_)],Y_test)
    print(f"Prediction using selected features using {model_} is :",score)
    accuracies_with_sfs[model]=score
    return sfs.k_feature_names_

In [129]:
selected_features=sf_selector(algo="LinearRegression()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    0.1s finished

[2023-02-01 16:02:39] Features: 1/62 -- score: 0.679844490313643[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.1s finished

[2023-02-01 16:02:39] Features: 2/62 -- score: 0.7560091134401516[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2023-02-01 16:02:39] Features: 3/62 -- score: 0.8182279341038639[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.2s finished

[2023-02-01 16:02:39] Features: 4/62 -- score: 0.8421545708346148[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n

Selected Features ('LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'LandContour', 'Neighborhood', 'Condition1', 'MasVnrType', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'KitchenQual', 'Functional', 'GarageType', 'SaleType')
Prediction using selected features using LinearRegression() is : 0.876824053780785


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-02-01 16:02:45] Features: 61/62 -- score: -1.7688938852202822e+21[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-01 16:02:45] Features: 62/62 -- score: -7.649652750287426e+20

In [130]:
fit_intercept = ["True","False"]
copy_X =["True","False"] 
positive=["False","True"]

random_grid = {'fit_intercept': fit_intercept,
               'positive': positive,
               'copy_X': copy_X}
print(random_grid)

{'fit_intercept': ['True', 'False'], 'positive': ['False', 'True'], 'copy_X': ['True', 'False']}


In [131]:
hyperparameter_tuning(algo="LinearRegression()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
LinearRegression() RandomCV Best Params : {'positive': 'False', 'fit_intercept': 'True', 'copy_X': 'True'}
LinearRegression() RandomCV Score: 0.8636738877114871
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.8636738877114871


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=100. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [11]:
# RF

In [132]:
selected_features=sf_selector(algo="RandomForestRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    6.0s finished

[2023-02-01 16:07:02] Features: 1/62 -- score: 0.7093234258882394[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    6.3s finished

[2023-02-01 16:07:09] Features: 2/62 -- score: 0.7537844909020834[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    6.4s finished

[2023-02-01 16:07:15] Features: 3/62 -- score: 0.7664695333447206[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  59 out o

Selected Features ('MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'BsmtFinSF1', 'TotalBsmtSF', 'GrLivArea', 'KitchenAbvGr', 'GarageCars', 'Street', 'Utilities', 'LandSlope', 'BldgType', 'ExterQual', 'BsmtQual', 'BsmtFinType1', 'CentralAir', 'KitchenQual', 'Functional', 'SaleCondition')
Prediction using selected features using RandomForestRegressor() is : 0.8877877845818886


In [133]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2,3,4, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['squared_error','absolute_error','friedman_mse','poisson']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 3, 4, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']}


In [134]:
hyperparameter_tuning(algo="RandomForestRegressor()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 5/5] END copy_X=True, fit_intercept=True, positive=False;, score=0.911 total time=   0.0s
[CV 1/5] END copy_X=True, fit_intercept=False, positive=False;, score=0.897 total time=   0.0s
[CV 1/5] END copy_X=False, fit_intercept=True, positive=False;, score=0.897 total time=   0.0s
[CV 2/5] END copy_X=False, fit_intercept=True, positive=False;, score=0.858 total time=   0.0s
[CV 5/5] END copy_X=False, fit_intercept=False, positive=False;, score=0.911 total time=   0.0s
[CV 1/5] END copy_X=False, fit_intercept=False, positive=True;, score=0.897 total time=   0.0s
[CV 4/5] END copy_X=True, fit_intercept=True, positive=False;, score=0.912 total time=   0.0s
[CV 2/5] END copy_X=True, fit_intercept=True, positive=False;, score=0.905 total time=   0.0s
[CV 2/5] END copy_X=True, fit_intercept=True, positive=True;, score=0.905 total time=   0.0s
[CV 3/5] END copy_X=True, fit_intercept=False, positive=False;, score=0.918 total time

In [135]:
# SVR

In [136]:
selected_features=sf_selector(algo="SVR()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    1.2s finished

[2023-02-01 16:19:51] Features: 1/62 -- score: -0.03601832875996762[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  61 | elapsed:    1.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    1.3s finished

[2023-02-01 16:19:53] Features: 2/62 -- score: -0.03691142544577834[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    1.2s finished

[2023-02-01 16:19:54] Features: 3/62 -- score: -0.037183690883610246[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    1.3s finished

[2023-02-01 16:19:55] Features: 4/62 -- score: -0.0373821596303

Selected Features ('OverallQual', 'YearBuilt', 'FullBath')
Prediction using selected features using SVR() is : -0.021348942417104055


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished

[2023-02-01 16:20:43] Features: 62/62 -- score: -0.039698167559629516

In [137]:
kernel = ["linear", "poly", "sigmoid","poly"]
degree = [int(x) for x in np.linspace(start = 1, stop = 20, num = 10)]
gamma= ["scale", "auto"]

tol=[1e-2]
max_iter=[100]


random_grid = {'kernel': kernel,
               'degree':degree,
               'max_iter':max_iter,
               'tol':tol,
               'gamma':gamma
               
               
               }
print(random_grid)

{'kernel': ['linear', 'poly', 'sigmoid', 'poly'], 'degree': [1, 3, 5, 7, 9, 11, 13, 15, 17, 20], 'max_iter': [100], 'tol': [0.01], 'gamma': ['scale', 'auto']}


In [138]:
hyperparameter_tuning(algo="SVR()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 80 is smaller than n_iter=100. Running 80 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 5/5] END criterion=absolute_error, max_depth=780, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=800;, score=0.889 total time=   4.1s
[CV 5/5] END criterion=absolute_error, max_depth=890, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1400;, score=0.858 total time=   5.4s
[CV 5/5] END criterion=poisson, max_depth=1000, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=1200;, score=0.891 total time=   1.9s
[CV 1/5] END criterion=poisson, max_depth=670, max_features=sqrt, min_samples_leaf=8, min_samples_split=5, n_estimators=200;, score=0.856 total time=   0.3s
[CV 2/5] END criterion=poisson, max_depth=670, max_features=sqrt, min_samples_leaf=8, min_samples_split=5, n_estimators=200;, score=0.873 total time=   0.3s
[CV 3/5] END criterion=poisson, max_depth=670, max_features=sqrt, min_samples_leaf=8, min_samples_split=5, n_estimators=200;, score=0.89

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-proce

[CV 4/5] END criterion=absolute_error, max_depth=890, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=1400;, score=0.860 total time=   5.4s
[CV 3/5] END criterion=squared_error, max_depth=450, max_features=sqrt, min_samples_leaf=6, min_samples_split=5, n_estimators=600;, score=0.901 total time=   0.8s
[CV 2/5] END criterion=poisson, max_depth=1000, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=1200;, score=0.913 total time=   1.9s
[CV 3/5] END criterion=poisson, max_depth=780, max_features=log2, min_samples_leaf=2, min_samples_split=14, n_estimators=2000;, score=0.908 total time=   2.8s
[CV 5/5] END criterion=poisson, max_depth=1000, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=800;, score=0.885 total time=   1.2s
[CV 1/5] END criterion=friedman_mse, max_depth=890, max_features=log2, min_samples_leaf=1, min_samples_split=14, n_estimators=2000;, score=0.869 total time=   3.0s
[CV 2/5] END criterion=friedman_

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-proce

SVR() RandomCV Best Params : {'tol': 0.01, 'max_iter': 100, 'kernel': 'poly', 'gamma': 'auto', 'degree': 20}
SVR() RandomCV Score: -1.724446512385569e-05
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  -1.724446512385569e-05


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-proce

In [139]:
#KNeighborsRegressor

In [140]:
selected_features=sf_selector(algo="KNeighborsRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    0.1s finished

[2023-02-01 16:20:44] Features: 1/62 -- score: 0.7011779774549705[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.1s finished

[2023-02-01 16:20:44] Features: 2/62 -- score: 0.7602949981612708[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2023-02-01 16:20:44] Features: 3/62 -- score: 0.8270914779752742[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.1s finished

[2023-02-01 16:20:44] Features: 4/62 -- score: 0.8523049161504801[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(

Selected Features ('OverallQual', 'YearBuilt', 'BsmtFinSF1', 'TotalBsmtSF', 'GrLivArea', 'Condition2', 'KitchenQual')
Prediction using selected features using KNeighborsRegressor() is : 0.8712205629550169


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2023-02-01 16:20:50] Features: 60/62 -- score: 0.842853820460655[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-02-01 16:20:50] Features: 61/62 -- score: 0.8437131950103266[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-01 16:20:50] Features: 62/62 -- score: 0.8332186315952939

In [141]:
weights=["uniform", "distance"]
n_neighbors = [int(x) for x in np.linspace(start = 5, stop = 350, num = 50)]
algorithm= ["auto", "ball_tree","kd_tree","brute"]
leaf_size=[int(x) for x in np.linspace(start = 5, stop = 20, num = 10)]
p = [1,2]


random_grid = {'weights': weights,
               'n_neighbors': n_neighbors,
               'algorithm': algorithm,
               'leaf_size':leaf_size,
               'p':p
               }
print(random_grid)

{'weights': ['uniform', 'distance'], 'n_neighbors': [5, 12, 19, 26, 33, 40, 47, 54, 61, 68, 75, 82, 89, 96, 103, 110, 117, 124, 131, 138, 145, 152, 159, 166, 173, 181, 188, 195, 202, 209, 216, 223, 230, 237, 244, 251, 258, 265, 272, 279, 286, 293, 300, 307, 314, 321, 328, 335, 342, 350], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': [5, 6, 8, 10, 11, 13, 15, 16, 18, 20], 'p': [1, 2]}


In [142]:
hyperparameter_tuning(algo="KNeighborsRegressor()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 4/5] END degree=1, gamma=auto, kernel=sigmoid, max_iter=100, tol=0.01;, score=-0.417 total time=   0.0s
[CV 3/5] END degree=3, gamma=scale, kernel=sigmoid, max_iter=100, tol=0.01;, score=-0.009 total time=   0.0s
[CV 4/5] END degree=3, gamma=scale, kernel=sigmoid, max_iter=100, tol=0.01;, score=-0.021 total time=   0.0s
[CV 5/5] END degree=3, gamma=scale, kernel=sigmoid, max_iter=100, tol=0.01;, score=-0.007 total time=   0.0s
[CV 1/5] END degree=3, gamma=scale, kernel=poly, max_iter=100, tol=0.01;, score=-0.147 total time=   0.0s
[CV 3/5] END degree=5, gamma=scale, kernel=sigmoid, max_iter=100, tol=0.01;, score=-0.009 total time=   0.0s
[CV 4/5] END degree=5, gamma=scale, kernel=sigmoid, max_iter=100, tol=0.01;, score=-0.021 total time=   0.0s
[CV 5/5] END degree=5, gamma=scale, kernel=sigmoid, max_iter=100, tol=0.01;, score=-0.007 total time=   0.0s
[CV 1/5] END degree=5, gamma=scale, kernel=poly, max_iter=100, tol=0.

In [143]:
# DecisionTreeRegressor

In [144]:
selected_features=sf_selector(algo="DecisionTreeRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    0.1s finished

[2023-02-01 16:20:51] Features: 1/62 -- score: 0.70646095151178[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.1s finished

[2023-02-01 16:20:51] Features: 2/62 -- score: 0.7502704460998386[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2023-02-01 16:20:51] Features: 3/62 -- score: 0.7637317537896153[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.1s finished

[2023-02-01 16:20:51] Features: 4/62 -- score: 0.7708405236196857[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_

Selected Features ('MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'BsmtFinSF1', 'TotalBsmtSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'YrSold', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition')
Prediction using selected features using DecisionTreeRegressor() is : 0.6965560365416512


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished

[2023-02-01 16:20:58] Features: 60/62 -- score: 0.778249944219682[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.1s finished

[2023-02-01 16:20:58] Features: 61/62 -- score: 0.7587019219645514[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished

[2023-02-01 16:20:59] Features: 62/62 -- score: 0.7476039444964682

In [145]:
splitter=["best", "random"]
# Number of features to consider at every split
max_features = ['sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,100)]
# Minimum number of samples required to split a node
min_samples_split = [2,3,4, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2, 4,6,8]
# Create the random grid
random_grid = {'splitter':splitter,
               'ccp_alpha':[1,0],
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['squared_error','absolute_error','friedman_mse','poisson']}
print(random_grid)

{'splitter': ['best', 'random'], 'ccp_alpha': [1, 0], 'max_features': ['sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000], 'min_samples_split': [2, 3, 4, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']}


In [146]:
hyperparameter_tuning(algo="DecisionTreeRegressor()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 2/5] END algorithm=ball_tree, leaf_size=13, n_neighbors=138, p=2, weights=uniform;, score=0.687 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=13, n_neighbors=138, p=2, weights=uniform;, score=0.745 total time=   0.0s
[CV 2/5] END algorithm=auto, leaf_size=13, n_neighbors=33, p=1, weights=distance;, score=0.845 total time=   0.0s
[CV 3/5] END algorithm=auto, leaf_size=13, n_neighbors=33, p=1, weights=distance;, score=0.869 total time=   0.0s
[CV 4/5] END algorithm=auto, leaf_size=13, n_neighbors=33, p=1, weights=distance;, score=0.863 total time=   0.0s
[CV 5/5] END algorithm=auto, leaf_size=13, n_neighbors=33, p=1, weights=distance;, score=0.853 total time=   0.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=11, n_neighbors=26, p=2, weights=distance;, score=0.842 total time=   0.0s
[CV 2/5] END algorithm=ball_tree, leaf_size=11, n_neighbors=26, p=2, weights=distance;, score=0.842 total time=   0.0s
[CV 3

In [147]:
#SGDRegressor

In [148]:
selected_features=sf_selector(algo="SGDRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    0.4s finished

[2023-02-01 16:21:00] Features: 1/62 -- score: 0.6795332925095348[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  61 | elapsed:    1.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    1.3s finished

[2023-02-01 16:21:01] Features: 2/62 -- score: 0.7542980811710336[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    1.2s finished

[2023-02-01 16:21:02] Features: 3/62 -- score: 0.8173373065555415[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  48 out of  59 | elapsed:    1.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    1.1s

Selected Features ('LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF', 'GrLivArea', 'GarageCars', 'Street', 'LandContour', 'Neighborhood', 'Condition1', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtExposure', 'Heating', 'KitchenQual', 'Functional', 'SaleType')
Prediction using selected features using SGDRegressor() is : 0.8724641699458564


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished

[2023-02-01 16:21:40] Features: 62/62 -- score: 0.8807834212589402

In [149]:
loss = ['squared_error','huber','epsilon_insensitive','squared_epsilon_insensitive']
penalty=['l2','l1','elasticnet','None']
alpha = [float(x) for x in np.linspace(0, 0.001,50)]
l1_ratio=[float(x) for x in np.linspace(0, 0.5,50)]
max_iter=[int(x) for x in np.linspace(8000, 20000,100)]
tol=[float(x) for x in np.linspace(0.01, 0.001,100)]
learning_rate=['constant','optimal','invscaling','adaptive']
eta0=[float(x) for x in np.linspace(0.1, 0.05,100)]
early_stopping = ['True','False']
validation_fraction=[float(x) for x in np.linspace(0.1, 0.9,50)]
n_iter_no_change = [1,2, 4,6,8,5,10]
average=['bool','int']
random_grid = {
               'loss': loss,
               'penalty': penalty,
               'alpha': alpha,
               'l1_ratio': l1_ratio,
              'max_iter':max_iter,
               'tol':tol,
                'learning_rate':learning_rate,
                'eta0':eta0,
               
                'validation_fraction':validation_fraction,
                'n_iter_no_change':n_iter_no_change
                }
print(random_grid)

{'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'], 'penalty': ['l2', 'l1', 'elasticnet', 'None'], 'alpha': [0.0, 2.0408163265306123e-05, 4.0816326530612245e-05, 6.122448979591836e-05, 8.163265306122449e-05, 0.00010204081632653062, 0.00012244897959183673, 0.00014285714285714287, 0.00016326530612244898, 0.0001836734693877551, 0.00020408163265306123, 0.00022448979591836734, 0.00024489795918367346, 0.0002653061224489796, 0.00028571428571428574, 0.0003061224489795918, 0.00032653061224489796, 0.0003469387755102041, 0.0003673469387755102, 0.0003877551020408163, 0.00040816326530612246, 0.00042857142857142855, 0.0004489795918367347, 0.00046938775510204083, 0.0004897959183673469, 0.0005102040816326531, 0.0005306122448979592, 0.0005510204081632653, 0.0005714285714285715, 0.0005918367346938776, 0.0006122448979591836, 0.0006326530612244898, 0.0006530612244897959, 0.000673469387755102, 0.0006938775510204082, 0.0007142857142857143, 0.0007346938775510204, 0.0007

In [150]:
hyperparameter_tuning(algo="SGDRegressor()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 2/5] END ccp_alpha=1, criterion=squared_error, max_depth=950, max_features=log2, min_samples_leaf=8, min_samples_split=5, splitter=best;, score=0.699 total time=   0.0s
[CV 3/5] END ccp_alpha=1, criterion=squared_error, max_depth=950, max_features=log2, min_samples_leaf=8, min_samples_split=5, splitter=best;, score=0.737 total time=   0.0s
[CV 4/5] END ccp_alpha=1, criterion=squared_error, max_depth=950, max_features=log2, min_samples_leaf=8, min_samples_split=5, splitter=best;, score=0.611 total time=   0.0s
[CV 5/5] END ccp_alpha=1, criterion=squared_error, max_depth=950, max_features=log2, min_samples_leaf=8, min_samples_split=5, splitter=best;, score=0.691 total time=   0.0s
[CV 1/5] END ccp_alpha=0, criterion=absolute_error, max_depth=760, max_features=log2, min_samples_leaf=4, min_samples_split=10, splitter=best;, score=0.816 total time=   0.0s
[CV 3/5] END ccp_alpha=0, criterion=absolute_error, max_depth=790, max

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_

[CV 2/5] END ccp_alpha=1, criterion=squared_error, max_depth=810, max_features=log2, min_samples_leaf=4, min_samples_split=10, splitter=best;, score=0.744 total time=   0.0s
[CV 3/5] END ccp_alpha=1, criterion=squared_error, max_depth=810, max_features=log2, min_samples_leaf=4, min_samples_split=10, splitter=best;, score=0.746 total time=   0.0s
[CV 4/5] END ccp_alpha=1, criterion=squared_error, max_depth=810, max_features=log2, min_samples_leaf=4, min_samples_split=10, splitter=best;, score=0.757 total time=   0.0s
[CV 5/5] END ccp_alpha=1, criterion=squared_error, max_depth=810, max_features=log2, min_samples_leaf=4, min_samples_split=10, splitter=best;, score=0.677 total time=   0.0s
[CV 1/5] END ccp_alpha=1, criterion=poisson, max_depth=820, max_features=sqrt, min_samples_leaf=6, min_samples_split=3, splitter=best;, score=0.727 total time=   0.0s
[CV 2/5] END ccp_alpha=1, criterion=poisson, max_depth=820, max_features=sqrt, min_samples_leaf=6, min_samples_split=3, splitter=best;, s

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV 5/5] END alpha=2.0408163265306123e-05, eta0=0.060606060606060615, l1_ratio=0.36734693877551017, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=16727, n_iter_no_change=6, penalty=l1, tol=0.0028181818181818178, validation_fraction=0.1;, score=0.886 total time=   0.1s
[CV 1/5] END alpha=0.00032653061224489796, eta0=0.05101010101010101, l1_ratio=0.22448979591836732, learning_rate=adaptive, loss=squared_error, max_iter=9454, n_iter_no_change=2, penalty=elasticnet, tol=0.008090909090909091, validation_fraction=0.1;, score=0.879 total time=   0.0s
[CV 2/5] END alpha=0.00032653061224489796, eta0=0.05101010101010101, l1_ratio=0.22448979591836732, learning_rate=adaptive, loss=squared_error, max_iter=9454, n_iter_no_change=2, penalty=elasticnet, tol=0.008090909090909091, validation_fraction=0.1;, score=0.904 total time=   0.0s
[CV 3/5] END alpha=0.00032653061224489796, eta0=0.05101010101010101, l1_ratio=0.22448979591836732, learning_rate=adaptive, loss=squared_error, max_i

...

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_

[CV 5/5] END alpha=0.0004489795918367347, eta0=0.07020202020202021, l1_ratio=0.4081632653061224, learning_rate=invscaling, loss=squared_epsilon_insensitive, max_iter=18060, n_iter_no_change=10, penalty=l2, tol=0.01, validation_fraction=0.2469387755102041;, score=0.882 total time=   0.0s
[CV 2/5] END alpha=0.0008163265306122449, eta0=0.09242424242424244, l1_ratio=0.13265306122448978, learning_rate=adaptive, loss=huber, max_iter=16606, n_iter_no_change=1, penalty=l2, tol=0.003363636363636363, validation_fraction=0.4755102040816327;, score=-1.655 total time=   0.9s
[CV 4/5] END alpha=0.0008163265306122449, eta0=0.09242424242424244, l1_ratio=0.13265306122448978, learning_rate=adaptive, loss=huber, max_iter=16606, n_iter_no_change=1, penalty=l2, tol=0.003363636363636363, validation_fraction=0.4755102040816327;, score=-1.934 total time=   0.9s
[CV 3/5] END alpha=0.0004897959183673469, eta0=0.09646464646464648, l1_ratio=0.4081632653061224, learning_rate=optimal, loss=epsilon_insensitive, max_

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor() RandomCV Best Params : {'validation_fraction': 0.4591836734693878, 'tol': 0.008, 'penalty': 'elasticnet', 'n_iter_no_change': 1, 'max_iter': 14787, 'loss': 'squared_epsilon_insensitive', 'learning_rate': 'adaptive', 'l1_ratio': 0.030612244897959183, 'eta0': 0.09898989898989899, 'alpha': 0.0007755102040816326}
SGDRegressor() RandomCV Score: 0.8740977646570636
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.8740829119789157


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradien

In [151]:
#ElasticNet

In [152]:
selected_features= sf_selector(algo="ElasticNet()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    0.1s finished

[2023-02-01 16:22:22] Features: 1/62 -- score: 0.13053016468007567[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.1s finished

[2023-02-01 16:22:22] Features: 2/62 -- score: 0.21728200829166813[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2023-02-01 16:22:22] Features: 3/62 -- score: 0.273145352695731[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.1s finished

[2023-02-01 16:22:22] Features: 4/62 -- score: 0.31458585776095516[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Paralle

Selected Features ('MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'OpenPorchSF', 'MoSold', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType')
Prediction using selected features using ElasticNet() is : 0.6335315363167291


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2023-02-01 16:22:27] Features: 60/62 -- score: 0.5977439640480293[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-02-01 16:22:27] Features: 61/62 -- score: 0.5976468198051526[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-02-01 16:22:27] Features: 62/62 -- score: 0.5975462295546231

In [153]:
alpha = [float(x) for x in np.linspace(0, 0.50,500)]
positive=['True','False']
l1_ratio=[float(x) for x in np.linspace(0, 0.5,100)]
max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
warm_start = ['True','False']
selection=['cyclic','random']
copy_X=['True','False']

random_grid = {'copy_X':copy_X,
               'alpha':alpha,
               'l1_ratio': l1_ratio,
               'max_iter':max_iter,
               'tol':tol,
                'warm_start':warm_start,
                'selection':selection
               
               
                }
print(random_grid)

{'copy_X': ['True', 'False'], 'alpha': [0.0, 0.001002004008016032, 0.002004008016032064, 0.0030060120240480957, 0.004008016032064128, 0.00501002004008016, 0.0060120240480961915, 0.007014028056112224, 0.008016032064128256, 0.009018036072144287, 0.01002004008016032, 0.011022044088176352, 0.012024048096192383, 0.013026052104208416, 0.014028056112224447, 0.01503006012024048, 0.01603206412825651, 0.017034068136272545, 0.018036072144288574, 0.019038076152304607, 0.02004008016032064, 0.02104208416833667, 0.022044088176352703, 0.023046092184368736, 0.024048096192384766, 0.0250501002004008, 0.026052104208416832, 0.027054108216432865, 0.028056112224448895, 0.029058116232464928, 0.03006012024048096, 0.03106212424849699, 0.03206412825651302, 0.033066132264529056, 0.03406813627254509, 0.03507014028056112, 0.03607214428857715, 0.03707414829659318, 0.038076152304609215, 0.03907815631262525, 0.04008016032064128, 0.041082164328657314, 0.04208416833667334, 0.04308617234468937, 0.044088176352705406, 0.04

In [154]:
hyperparameter_tuning(algo="ElasticNet()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 5/5] END alpha=0.0, eta0=0.0691919191919192, l1_ratio=0.44897959183673464, learning_rate=invscaling, loss=huber, max_iter=12121, n_iter_no_change=10, penalty=elasticnet, tol=0.006454545454545454, validation_fraction=0.5408163265306123;, score=-8.161 total time=   1.3s
[CV 3/5] END alpha=0.0009183673469387755, eta0=0.05050505050505051, l1_ratio=0.0510204081632653, learning_rate=adaptive, loss=squared_error, max_iter=13818, n_iter_no_change=10, penalty=l1, tol=0.009727272727272727, validation_fraction=0.573469387755102;, score=0.926 total time=   1.3s
[CV 2/5] END alpha=0.0005306122448979592, eta0=0.08383838383838385, l1_ratio=0.0510204081632653, learning_rate=constant, loss=squared_epsilon_insensitive, max_iter=15151, n_iter_no_change=1, penalty=l1, tol=0.004818181818181818, validation_fraction=0.6551020408163265;, score=0.854 total time=   0.0s
[CV 3/5] END alpha=0.0005306122448979592, eta0=0.08383838383838385, l1_ratio

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.991e+11, tolerance: 2.646e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.822e+11, tolerance: 2.616e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.

[CV 2/5] END alpha=0.29058116232464926, copy_X=False, l1_ratio=0.11616161616161617, max_iter=5537, selection=cyclic, tol=0.003216080402010049, warm_start=True;, score=0.719 total time=   0.0s
[CV 3/5] END alpha=0.29058116232464926, copy_X=False, l1_ratio=0.11616161616161617, max_iter=5537, selection=cyclic, tol=0.003216080402010049, warm_start=True;, score=0.758 total time=   0.0s
[CV 4/5] END alpha=0.29058116232464926, copy_X=False, l1_ratio=0.11616161616161617, max_iter=5537, selection=cyclic, tol=0.003216080402010049, warm_start=True;, score=0.691 total time=   0.0s
[CV 5/5] END alpha=0.29058116232464926, copy_X=False, l1_ratio=0.11616161616161617, max_iter=5537, selection=cyclic, tol=0.003216080402010049, warm_start=True;, score=0.684 total time=   0.0s
[CV 1/5] END alpha=0.002004008016032064, copy_X=True, l1_ratio=0.09595959595959597, max_iter=7909, selection=cyclic, tol=0.004391959798994974, warm_start=True;, score=0.878 total time=   0.0s
[CV 2/5] END alpha=0.002004008016032064,

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.703e+11, tolerance: 2.575e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.950e+11, tolerance: 2.692e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.

ElasticNet() RandomCV Best Params : {'warm_start': 'False', 'tol': 0.006562814070351758, 'selection': 'cyclic', 'max_iter': 6743, 'l1_ratio': 0.14141414141414144, 'copy_X': 'True', 'alpha': 0.002004008016032064}
ElasticNet() RandomCV Score: 0.8788561761481258
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.8788561761481258


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.274e+11, tolerance: 7.604e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.014e+11, tolerance: 7.274e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.

In [155]:
#Lasso

In [156]:
selected_features=sf_selector(algo="Lasso()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    0.2s finished

[2023-02-01 16:22:28] Features: 1/62 -- score: 0.6798454456419083[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.1s finished

[2023-02-01 16:22:29] Features: 2/62 -- score: 0.7560099049949636[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2023-02-01 16:22:29] Features: 3/62 -- score: 0.8182278859860217[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.1s finished

[2023-02-01 16:22:29] Features: 4/62 -- score: 0.842154301510728[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n

Selected Features ('LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'LandContour', 'Neighborhood', 'Condition1', 'MasVnrType', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'KitchenQual', 'Functional', 'GarageType', 'SaleType')
Prediction using selected features using Lasso() is : 0.8768979354777238


In [157]:
max_iter=[int(x) for x in np.linspace(2000, 10000,500)]
tol=[float(x) for x in np.linspace(0.01, 0.001,300)]
warm_start = ['True','False']
selection=['cyclic','random']
alpha = [float(x) for x in np.linspace(0,0.50,500)]
# positive=['True','False']
copy_X=['True','False']
random_grid = {
               'alpha':alpha,
              
               'max_iter':max_iter,
               'tol':tol,
                'positive':positive,
                'copy_X':copy_X,
                'warm_start':warm_start,
                'selection':selection
               
               
                }
print(random_grid)

{'alpha': [0.0, 0.001002004008016032, 0.002004008016032064, 0.0030060120240480957, 0.004008016032064128, 0.00501002004008016, 0.0060120240480961915, 0.007014028056112224, 0.008016032064128256, 0.009018036072144287, 0.01002004008016032, 0.011022044088176352, 0.012024048096192383, 0.013026052104208416, 0.014028056112224447, 0.01503006012024048, 0.01603206412825651, 0.017034068136272545, 0.018036072144288574, 0.019038076152304607, 0.02004008016032064, 0.02104208416833667, 0.022044088176352703, 0.023046092184368736, 0.024048096192384766, 0.0250501002004008, 0.026052104208416832, 0.027054108216432865, 0.028056112224448895, 0.029058116232464928, 0.03006012024048096, 0.03106212424849699, 0.03206412825651302, 0.033066132264529056, 0.03406813627254509, 0.03507014028056112, 0.03607214428857715, 0.03707414829659318, 0.038076152304609215, 0.03907815631262525, 0.04008016032064128, 0.041082164328657314, 0.04208416833667334, 0.04308617234468937, 0.044088176352705406, 0.04509018036072144, 0.0460921843

In [158]:
hyperparameter_tuning(algo="Lasso()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 2/5] END alpha=0.0005714285714285715, eta0=0.05656565656565657, l1_ratio=0.061224489795918366, learning_rate=optimal, loss=epsilon_insensitive, max_iter=10545, n_iter_no_change=10, penalty=l2, tol=0.007909090909090909, validation_fraction=0.7367346938775511;, score=-5.891 total time=   0.6s
[CV 1/5] END alpha=0.0007755102040816326, eta0=0.09898989898989899, l1_ratio=0.030612244897959183, learning_rate=adaptive, loss=squared_epsilon_insensitive, max_iter=14787, n_iter_no_change=1, penalty=elasticnet, tol=0.008, validation_fraction=0.4591836734693878;, score=0.877 total time=   0.0s
[CV 3/5] END alpha=0.375751503006012, copy_X=True, l1_ratio=0.3484848484848485, max_iter=3005, selection=random, tol=0.004165829145728642, warm_start=False;, score=0.765 total time=   0.0s
[CV 4/5] END alpha=0.024048096192384766, copy_X=False, l1_ratio=0.12121212121212122, max_iter=2321, selection=cyclic, tol=0.009683417085427136, warm_start=T

In [159]:
# Ridge

In [160]:
selected_features=sf_selector(algo="Ridge()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    0.1s finished

[2023-02-01 16:22:36] Features: 1/62 -- score: 0.6776702727966265[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.1s finished

[2023-02-01 16:22:36] Features: 2/62 -- score: 0.7548308929184657[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2023-02-01 16:22:36] Features: 3/62 -- score: 0.817083647269284[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.1s finished

[2023-02-01 16:22:36] Features: 4/62 -- score: 0.8413284453405584[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n

Selected Features ('LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'Street', 'LandContour', 'Neighborhood', 'Condition1', 'MasVnrType', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'GarageType', 'SaleType')
Prediction using selected features using Ridge() is : 0.875279031226215


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished

[2023-02-01 16:22:41] Features: 55/62 -- score: 0.8958429392986638[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished

[2023-02-01 16:22:41] Features: 56/62 -- score: 0.8953674495181028[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.0s finished

[2023-02-01 16:22:41] Features: 57/62 -- score: 0.8949155408692624[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished

[2023-02-01 16:22:41] Features: 58/62 -- sc

In [161]:
max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
alpha = [float(x) for x in np.linspace(0.1,0.50,400)]
positive=['False']
copy_X=['True','False']
solver=["auto"]

random_grid = {
               'alpha':alpha,
              
               'max_iter':max_iter,
               'tol':tol,
                'positive':positive,
                'copy_X':copy_X,
                'solver':solver
               
               
                }
print(random_grid)

{'alpha': [0.1, 0.10100250626566416, 0.10200501253132832, 0.10300751879699249, 0.10401002506265665, 0.10501253132832081, 0.10601503759398496, 0.10701754385964912, 0.1080200501253133, 0.10902255639097745, 0.11002506265664161, 0.11102756892230577, 0.11203007518796992, 0.1130325814536341, 0.11403508771929825, 0.11503759398496241, 0.11604010025062657, 0.11704260651629073, 0.1180451127819549, 0.11904761904761905, 0.12005012531328321, 0.12105263157894737, 0.12205513784461153, 0.1230576441102757, 0.12406015037593986, 0.12506265664160401, 0.12606516290726819, 0.12706766917293233, 0.1280701754385965, 0.12907268170426067, 0.13007518796992482, 0.131077694235589, 0.13208020050125313, 0.1330827067669173, 0.13408521303258147, 0.13508771929824562, 0.1360902255639098, 0.13709273182957393, 0.1380952380952381, 0.13909774436090228, 0.14010025062656642, 0.1411027568922306, 0.14210526315789473, 0.1431077694235589, 0.14411027568922308, 0.14511278195488722, 0.1461152882205514, 0.14711779448621554, 0.14812030

In [162]:
hyperparameter_tuning(algo="Ridge()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train.loc[:,list(selected_features)],Y_train=Y_train,X_test=X_test.loc[:,list(selected_features)],Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 5/5] END alpha=0.028056112224448895, copy_X=False, max_iter=3154, positive=False, selection=cyclic, tol=0.0016923076923076909, warm_start=False;, score=0.883 total time=   0.0s
[CV 1/5] END alpha=0.17434869739478956, copy_X=False, max_iter=6665, positive=True, selection=cyclic, tol=0.00777257525083612, warm_start=True;, score=0.881 total time=   0.0s
[CV 2/5] END alpha=0.17434869739478956, copy_X=False, max_iter=6665, positive=True, selection=cyclic, tol=0.00777257525083612, warm_start=True;, score=0.905 total time=   0.0s
[CV 3/5] END alpha=0.12124248496993988, copy_X=True, max_iter=4468, positive=True, selection=random, tol=0.0026254180602006683, warm_start=False;, score=0.918 total time=   0.0s
[CV 4/5] END alpha=0.12124248496993988, copy_X=True, max_iter=4468, positive=True, selection=random, tol=0.0026254180602006683, warm_start=False;, score=0.892 total time=   0.0s
[CV 5/5] END alpha=0.12124248496993988, copy_X=T

In [163]:
accuracies_with_hyperparameter_tuning

{'LinearRegression()': 0.8636738877114871,
 'RandomForestRegressor()': 0.9027404283164311,
 'SVR()': -1.724446512385569e-05,
 'KNeighborsRegressor()': 0.8713196886668324,
 'DecisionTreeRegressor()': 0.7306322588578171,
 'SGDRegressor()': 0.8740829119789157,
 'ElasticNet()': 0.8788561761481258,
 'Lasso()': 0.8636966761305713,
 'Ridge()': 0.8632007478818366}

In [ ]:
accuracies_with_sfs

In [229]:
# RandomForestRegressor

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 3, 4, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']}


In [231]:
# hyperparameter_tuning(algo="RandomForestRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=3,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
RandomForestRegressor() RandomCV Best Params : {'n_estimators': 200, 'min_samples_split': 14, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 890, 'criterion': 'friedman_mse'}
RandomForestRegressor() RandomCV Score: 0.8154736655041288
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.8201522653525275


In [236]:
# 3. SVR

In [237]:
# kernel = ["linear", "poly", "sigmoid","poly"]
# degree = [int(x) for x in np.linspace(start = 1, stop = 20, num = 10)]
# gamma= ["scale", "auto"]

# tol=[1e-2]
# max_iter=[100]


# random_grid = {'kernel': kernel,
#                'degree':degree,
#                'max_iter':max_iter,
#                'tol':tol,
#                'gamma':gamma
               
               
#                }
# print(random_grid)

{'kernel': ['linear', 'poly', 'sigmoid', 'poly'], 'degree': [1, 3, 5, 7, 9, 11, 13, 15, 17, 20], 'max_iter': [100], 'tol': [0.01], 'gamma': ['scale', 'auto']}


In [13]:
# hyperparameter_tuning(algo="SVR()",param_grid=random_grid,n_iter=10,cv=5,verbose=0,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

In [244]:
# 4 KNN Regressor

In [245]:
# weights=["uniform", "distance"]
# n_neighbors = [int(x) for x in np.linspace(start = 5, stop = 350, num = 50)]
# algorithm= ["auto", "ball_tree","kd_tree","brute"]
# leaf_size=[int(x) for x in np.linspace(start = 5, stop = 20, num = 10)]
# p = [1,2]


# random_grid = {'weights': weights,
#                'n_neighbors': n_neighbors,
#                'algorithm': algorithm,
#                'leaf_size':leaf_size,
#                'p':p
#                }
# print(random_grid)

{'weights': ['uniform', 'distance'], 'n_neighbors': [5, 12, 19, 26, 33, 40, 47, 54, 61, 68, 75, 82, 89, 96, 103, 110, 117, 124, 131, 138, 145, 152, 159, 166, 173, 181, 188, 195, 202, 209, 216, 223, 230, 237, 244, 251, 258, 265, 272, 279, 286, 293, 300, 307, 314, 321, 328, 335, 342, 350], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': [5, 6, 8, 10, 11, 13, 15, 16, 18, 20], 'p': [1, 2]}


In [15]:
# hyperparameter_tuning(algo="KNeighborsRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

In [16]:
# model=KNeighborsRegressor()
# knn_random_cv=RandomizedSearchCV(estimator=model,param_distributions=random_grid,n_iter=100,cv=5,verbose=1,random_state=100,n_jobs=-1)
# knn_random_cv.fit(X_train,Y_train)
# best_params_=knn_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=knn_random_cv.best_estimator_
# y_pred4=best_random_grid.predict(X_test)
# score_4=r2_score(Y_test,y_pred4)
# print("RandomCV Score: ",score_4)

In [17]:
# param_grid4=defaultdict(list)
# for val in best_params_:
#     param_grid4[val].append(best_params_[val])
# print(param_grid4)

In [18]:
# model=KNeighborsRegressor()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid4,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)

In [19]:
# best_grid2=grid_search.best_estimator_
# final=best_grid2.predict(X_test)
# score_final4=r2_score(Y_test,final)
# print(score_final4)
# accuracies["KNeighborsRegressor"]=score_final4

In [20]:
## 5 DecisionTreeRegressor

In [21]:
# splitter=["best", "random"]
# # Number of features to consider at every split
# max_features = ['sqrt','log2']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 1000,100)]
# # Minimum number of samples required to split a node
# min_samples_split = [2,3,4, 5, 10,14]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1,2, 4,6,8]
# # Create the random grid
# random_grid = {'splitter':splitter,
#                'ccp_alpha':[1,0],
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#               'criterion':['squared_error','absolute_error','friedman_mse','poisson']}
# print(random_grid)

NameError: name 'np' is not defined

In [22]:
# hyperparameter_tuning(algo="DecisionTreeRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

In [254]:
# dt=DecisionTreeRegressor()
# dt_random_cv=RandomizedSearchCV(estimator=dt,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# dt_random_cv.fit(X_train,Y_train)

# best_params_=dt_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=dt_random_cv.best_estimator_
# y_pred5=best_random_grid.predict(X_test)
# score_5=r2_score(Y_test,y_pred5)
# print("RandomCV Score: ",score_5)

In [255]:
# param_grid5=defaultdict(list)
# for val in best_params_:
#     param_grid5[val].append(best_params_[val])
# print(param_grid5)

In [256]:
# model=DecisionTreeRegressor()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid5,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final5=best_grid2.predict(X_test)
# score_final5=r2_score(Y_test,final5)
# print(score_final5)
# accuracies["DecisionTreeRegressor"]=score_final5

In [257]:
## 6 SGDRegressor

In [23]:
# loss = ['squared_error','huber','epsilon_insensitive','squared_epsilon_insensitive']
# penalty=['l2','l1','elasticnet','None']
# alpha = [float(x) for x in np.linspace(0, 0.001,50)]
# l1_ratio=[float(x) for x in np.linspace(0, 0.5,50)]
# max_iter=[int(x) for x in np.linspace(8000, 20000,100)]
# tol=[float(x) for x in np.linspace(0.01, 0.001,100)]
# learning_rate=['constant','optimal','invscaling','adaptive']
# eta0=[float(x) for x in np.linspace(0.1, 0.05,100)]
# early_stopping = ['True','False']
# validation_fraction=[float(x) for x in np.linspace(0.1, 0.9,50)]
# n_iter_no_change = [1,2, 4,6,8,5,10]
# average=['bool','int']
# random_grid = {
#                'loss': loss,
#                'penalty': penalty,
#                'alpha': alpha,
#                'l1_ratio': l1_ratio,
#               'max_iter':max_iter,
#                'tol':tol,
#                 'learning_rate':learning_rate,
#                 'eta0':eta0,
               
#                 'validation_fraction':validation_fraction,
#                 'n_iter_no_change':n_iter_no_change
#                 }
# print(random_grid)

In [24]:
# hyperparameter_tuning(algo="SGDRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

In [260]:
# sgd=SGDRegressor()
# sgd_random_cv=RandomizedSearchCV(estimator=sgd,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# sgd_random_cv.fit(X_train,Y_train)

# best_params_=sgd_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=sgd_random_cv.best_estimator_
# y_pred6=best_random_grid.predict(X_test)
# score_6=r2_score(Y_test,y_pred6)
# print("RandomCV Score: ",score_6)

In [261]:
# param_grid6=defaultdict(list)
# for val in best_params_:
#     param_grid6[val].append(best_params_[val])
# print(param_grid6)

In [262]:
# model=SGDRegressor()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid6,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final6=best_grid2.predict(X_test)
# score_final6=r2_score(Y_test,final6)
# print(score_final6)
# accuracies["SGDRegressor"]=score_final6

In [264]:
# 7 ElasticNet

In [25]:
# alpha = [float(x) for x in np.linspace(0, 0.50,500)]
# positive=['True','False']
# l1_ratio=[float(x) for x in np.linspace(0, 0.5,100)]
# max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
# tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
# warm_start = ['True','False']
# selection=['cyclic','random']
# copy_X=['True','False']

# random_grid = {'copy_X':copy_X,
#                'alpha':alpha,
#                'l1_ratio': l1_ratio,
#                'max_iter':max_iter,
#                'tol':tol,
#                 'warm_start':warm_start,
#                 'selection':selection
               
               
#                 }
# print(random_grid)

In [26]:
# hyperparameter_tuning(algo="ElasticNet()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

In [267]:
# en=ElasticNet()
# en_random_cv=RandomizedSearchCV(estimator=en,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# en_random_cv.fit(X_train,Y_train)

# best_params_=en_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=en_random_cv.best_estimator_
# y_pred7=best_random_grid.predict(X_test)
# score_7=r2_score(Y_test,y_pred7)
# print("RandomCV Score: ",score_7)

In [268]:
# param_grid7=defaultdict(list)
# for val in best_params_:
#     param_grid7[val].append(best_params_[val])
# print(param_grid7)

In [269]:
# model=ElasticNet()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid7,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final7=best_grid2.predict(X_test)
# score_final7=r2_score(Y_test,final7)
# print(score_final7)
# accuracies["ElasticNet"]=score_final7

In [270]:
# 8 Lasso

In [271]:
# alpha = [float(x) for x in np.linspace(0, 0.50,500)]
# positive=['True','False']
# l1_ratio=[float(x) for x in np.linspace(0, 0.5,100)]
# max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
# tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
# warm_start = ['True','False']
# selection=['cyclic','random']
# copy_X=['True','False']

# random_grid = {'copy_X':copy_X,
#                'alpha':alpha,
#                'l1_ratio': l1_ratio,
#                'max_iter':max_iter,
#                'tol':tol,
#                 'warm_start':warm_start,
#                 'selection':selection
               
               
#                 }
# print(random_grid)

In [27]:
# max_iter=[int(x) for x in np.linspace(2000, 10000,500)]
# tol=[float(x) for x in np.linspace(0.01, 0.001,300)]
# warm_start = ['True','False']
# selection=['cyclic','random']
# alpha = [float(x) for x in np.linspace(0,0.50,500)]
# # positive=['True','False']
# copy_X=['True','False']
# random_grid = {
#                'alpha':alpha,
              
#                'max_iter':max_iter,
#                'tol':tol,
#                 'positive':positive,
#                 'copy_X':copy_X,
#                 'warm_start':warm_start,
#                 'selection':selection
               
               
#                 }
# print(random_grid)


In [28]:
# hyperparameter_tuning(algo="Lasso()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

In [274]:
# lasso=Lasso()
# lasso_random_cv=RandomizedSearchCV(estimator=lasso,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# lasso_random_cv.fit(X_train,Y_train)

# best_params_=lasso_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=lasso_random_cv.best_estimator_
# y_pred8=best_random_grid.predict(X_test)
# score_8=r2_score(Y_test,y_pred8)
# print("RandomCV Score: ",score_8)

In [275]:
# param_grid8=defaultdict(list)
# for val in best_params_:
#     param_grid8[val].append(best_params_[val])
# print(param_grid8)

In [276]:
# param_grid8

In [277]:
# model=Lasso()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid8,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final8=best_grid2.predict(X_test)
# score_final8=r2_score(Y_test,final8)
# print(score_final8)
# accuracies["Lasso"]=score_final8

In [278]:
# 9 Ridge

In [29]:
# max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
# tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
# alpha = [float(x) for x in np.linspace(0.1,0.50,400)]
# positive=['False']
# copy_X=['True','False']
# solver=["auto"]

# random_grid = {
#                'alpha':alpha,
              
#                'max_iter':max_iter,
#                'tol':tol,
#                 'positive':positive,
#                 'copy_X':copy_X,
#                 'solver':solver
               
               
#                 }
# print(random_grid)

In [30]:
# hyperparameter_tuning(algo="Ridge()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

In [281]:
# ridge=Ridge()
# ridge_random_cv=RandomizedSearchCV(estimator=ridge,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# ridge_random_cv.fit(X_train,Y_train)

# best_params_=ridge_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=ridge_random_cv.best_estimator_
# y_pred9=best_random_grid.predict(X_test)
# score_9=r2_score(Y_test,y_pred9)
# print("RandomCV Score: ",score_9)

In [282]:
# param_grid9=defaultdict(list)
# for val in best_params_:
#     param_grid9[val].append(best_params_[val])
# print(param_grid9)

In [283]:
# model=Ridge()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid9,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final9=best_grid2.predict(X_test)
# score_final9=r2_score(Y_test,final9)
# print(score_final9)
# accuracies["Ridge"]=score_final9

In [ ]:

# sfs1 = SFS(RandomForestRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=RandomForestRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["RandomForestRegressor"]=score

In [ ]:
# sfs1 = SFS(SVR(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=1,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=SVR()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SVR"]=score

In [ ]:
# sfs1 = SFS(KNeighborsRegressor(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=1,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=KNeighborsRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["KNeighborsRegressor"]=score

In [ ]:
# sfs1 = SFS(DecisionTreeRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=DecisionTreeRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["DecisionTreeRegressor"]=score

In [ ]:
# sfs1 = SFS(SGDRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=SGDRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SGDRegressor"]=score

In [174]:
# sfs1 = SFS(ElasticNet(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=ElasticNet()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["ElasticNet"]=score

In [214]:
# sfs1 = SFS(Lasso(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=Lasso()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Lasso"]=score

In [ ]:
# sfs1 = SFS(Ridge(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=Ridge()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Ridge"]=score